Коллаборативная рекомендательная система item-based

Подготовка данных

 Файл movie.csvmovieId,title,genres   
 Класс - Movies  
 Атрибуты:  
 file_path - путь до файл а с данными  
 data - данные из файла  
 Методы:  
 read_file - считывает data из файла  
 read_title - получение title фильма для ссылки из файла links.csv  
 read_name - парсер названия по ссылке  
 remove_errors  
 План анализа:  
 -% ошибок-исправление ошибок для 1000 строк, извлечение года  
 -создание файла с исправленными данными  


In [879]:
import importlib

import data_utils
importlib.reload(data_utils)

<module 'data_utils' from '/Users/ekaterinagoryacheva/Desktop/rec_sys_CF_item/data_utils.py'>

In [880]:
import pytest
pytest.main(['data_utils.py'])

============================= test session starts ==============================
platform darwin -- Python 3.11.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /Users/ekaterinagoryacheva/Desktop/rec_sys_CF_item
plugins: anyio-4.3.0
collected 4 items

data_utils.py ....                                                       [100%]

============================== 4 passed in 0.09s ===============================


<ExitCode.OK: 0>

In [881]:
from data_utils import Movies, read_title, read_name

Данные в исходном файле  
% ошибок  
Исправление ошибок для 1000 строк, извлечение года + создание файла с исправленными данными  
Примечание:  
Ошибки содержат те строки, где названия фильмов обособлены кавычками  
Формат ошибки  
-неправильный порядок слов в названии или не указан год  
С помощью links.csv, movie.csv и наличия movieID в обоих файлах нахожу ссылку на фильм, и если есть ошибка в названии - парсером считываю название с сайта

In [882]:
data_movie = Movies('./data/movies.csv')
data_movie.read_file()

for movie in data_movie.data[28:60:8]:
    print(movie[:-1])

len(data_movie.data)

29,"City of Lost Children, The (Cité des enfants perdus, La) (1995)",Adventure|Drama|Fantasy|Mystery|Sci-Fi
40,"Cry, the Beloved Country (1995)",Drama
48,Pocahontas (1995),Animation|Children|Drama|Musical|Romance
58,"Postman, The (Postino, Il) (1994)",Comedy|Drama|Romance


9742

In [883]:
percent_error = data_movie.count_errors("error.txt")
print(f"{percent_error:.2f}% of lines in data - incorrect\n")

22.72% of lines in data - incorrect



Реализация алгоритма

In [884]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [885]:
movies = pd.read_csv('./data/movies.csv')
ratings = pd.read_csv('./data/ratings.csv')

In [886]:
movies.drop(columns=['genres'], axis = 1, inplace = True)
ratings.drop(['timestamp'], axis = 1, inplace = True)

In [887]:
user_item_matrix = ratings.pivot(index = 'movieId', columns = 'userId', values = 'rating')
user_item_matrix.fillna(0, inplace = True)
user_item_matrix.head(5)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [888]:
user_item_matrix.shape

(9724, 610)

In [889]:
# колво оценок от каждого пользователя
users_votes = ratings.groupby('userId')['rating'].agg('count')

# колво оценок у каждого фильма
movies_votes = ratings.groupby('movieId')['rating'].agg('count')

In [890]:
top_votes = users_votes.sort_values(ascending=False)[:3]
bottom_votes = users_votes.sort_values()[:3]

print(pd.concat([top_votes, bottom_votes]))
print(f'median: {users_votes.median()}\n' \
      f'mode: {users_votes.mode()[0]}')

userId
414    2698
599    2478
474    2108
442      20
406      20
147      20
Name: rating, dtype: int64
median: 70.5
mode: 21


In [891]:
top_votes = movies_votes.sort_values(ascending=False)[:3]
bottom_votes = movies_votes.sort_values()[:3]

print(pd.concat([top_votes, bottom_votes]))
print(f'median: {movies_votes.median()}\n' \
      f'mode: {movies_votes.mode()[0]}')

movieId
356       329
318       317
296       307
193609      1
4032        1
57526       1
Name: rating, dtype: int64
median: 3.0
mode: 1


In [892]:
user_mask = users_votes[users_votes > 30].index
movie_mask = movies_votes[movies_votes > 10].index
 
user_item_matrix = user_item_matrix.loc[movie_mask,:]
user_item_matrix = user_item_matrix.loc[:,user_mask]

In [893]:
user_item_matrix.shape

(2121, 498)

In [894]:
csr_data = csr_matrix(user_item_matrix.values)

In [895]:
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,movieId,1,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,1,4.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
1,2,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
2,3,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,5,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0


In [896]:
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute', #полный перебор
                       n_neighbors = 20,
                       n_jobs = -1)
 
knn.fit(csr_data) 

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [897]:
recommendations = 10
search_word = 'Toy Story'

movie_search = movies[movies['title'].str.contains(search_word)]
movie_search

,movieId,title
0,1,Toy Story (1995)
2355,3114,Toy Story 2 (1999)
7355,78499,Toy Story 3 (2010)


In [898]:
movie_id = movie_search.iloc[0]['movieId']
movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]

In [899]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = recommendations + 1)

In [900]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()
indices_distances = list(zip(indices_list, distances_list))
 
print(indices_list[:3])
print(distances_list[:3])
print(indices_distances[:3])

[0, 217, 1048]
[0.0, 0.39532165904288463, 0.4067677702555277]
[(0, 0.0), (217, 0.39532165904288463), (1048, 0.4067677702555277)]


In [901]:
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)

indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(217, 0.39532165904288463),
 (1048, 0.4067677702555277),
 (124, 0.4127717548328176),
 (169, 0.4154323721626022),
 (308, 0.43248596082180935),
 (454, 0.4345457909576894),
 (173, 0.43974638168369784),
 (1292, 0.4427476930498869),
 (67, 0.4439348632647955),
 (504, 0.444752602440587)]

In [902]:
recom_list = []
 
for id_dist in indices_distances_sorted:
    movie_id = user_item_matrix.iloc[id_dist[0]]['movieId']
    
    title = read_title(str(int(movie_id)))
    name = read_name(title)
    dist = id_dist[1]

    recom_list.append({'Title' : name, 'Distance' : dist})

In [903]:
recom_df = pd.DataFrame(recom_list, index = range(1, recommendations + 1))
recom_df

,Title,Distance
1,Jurassic Park,0.395322
2,Toy Story 2,0.406768
3,Star Wars,0.412772
4,Forrest Gump,0.415432
5,Independence Day,0.432486
6,Star Wars: Episode VI - Return of the Jedi,0.434546
7,The Lion King,0.439746
8,Shrek,0.442748
9,Apollo 13,0.443935
10,Back to the Future,0.444753
